## 更新约束矩阵

为了实现Graph SLAM，我们引入了一个矩阵与一个向量，分别为omega和xi。该矩阵是方形的，并标有所有机器人姿势（xi）和所有地标（Li）。例如，在每次进行观察中，当你在两个姿势之间移动某个距离`dx`并且可以将这两个位置关联起来时，可以将其表示为这些矩阵中的数值关系。

下面，你看到的是一个omega的矩阵表示与一个xi的向量表示。

<img src='images/omega_xi.png' width=20% height=20% />


### 求解x，L

为了“确定”所有这些姿势和地标位置，我们可以使用线性代数。所有位置值都在向量`mu`中，该向量可以计算为omega乘以xi的倒数的乘积。

---


## 约束更新

在下面的代码中，我们构建了`omega` 和`xi`约束矩阵，并根据地标传感器测量和运动数据来更新这些矩阵。

#### 传感器测量

当感测到姿势和地标之间的距离`dl`时，更新约束矩阵，如下所示：
* 在 `xt`和`l`交叉点的索引处将 `[[1, -1], [-1, 1]]`添加到omega中
* 在`xt`和`l`行中将 `-dl`  和`dl`添加到xi中

值2表示测量的“强度”，而不是值1。

你将看到三个新的`dl`值作为函数`Z0, Z1, Z2`的新输入，如下所示。

#### 运动
当机器人移动一定量`dx`时，更新约束矩阵时，如下所示：
* 在`xt` 和 `xt+1` 交叉点的索引处将`[[1, -1], [-1, 1]]`添加到omega中
* 在`xt` 和 `xt+1` 的行中将`-dx` 和 `dx` 添加到xi中

## 更改代码，使最后一次传感器测量（Z2）的置信度非常高。

将该测量的“强度”乘以5倍。

In [ ]:
import numpy as np


def mu_from_positions(initial_pos, move1, move2, Z0, Z1, Z2):
    
    ## TODO: construct constraint matrices
    ## and add each position/motion constraint to them
    
    # initialize constraint matrices with 0's
    # Now these are 4x4 because of 3 poses and a landmark
    omega = np.zeros((4,4))
    xi = np.zeros((4,1))
    
    # add initial pose constraint
    omega[0][0] = 1
    xi[0] = initial_pos
    
    # account for the first motion, dx = move1
    omega += [[1., -1., 0., 0.],
              [-1., 1., 0., 0.],
              [0., 0., 0., 0.],
              [0., 0., 0., 0.]]
    xi += [[-move1],
           [move1],
           [0.],
           [0.]]
    
    # account for the second motion
    omega += [[0., 0., 0., 0.],
              [0., 1., -1., 0.],
              [0., -1., 1., 0.],
              [0., 0., 0., 0.]]
    xi += [[0.],
           [-move2],
           [move2],
           [0.]]
    
    
    ## TODO: Include three new sensor measurements for the landmark, L
    # incorporate first sense
    omega += [[1., 0., 0., -1.],
              [0., 0., 0., 0.],
              [0., 0., 0., 0.], 
              [-1., 0., 0., 1.]]
    xi += [[-Z0],
           [0.0],
           [0.0],
           [Z0]]

    # incorporate second sense
    omega += [[0., 0., 0., 0.],
              [0., 1., 0., -1.],
              [0., 0., 0., 0.], 
              [0., -1., 0., 1.]]
    xi += [[0.],
           [-Z1],
           [0.],
           [Z1]]
    
    ## This third sense is now *very confident* and
    ## we multiply everything by a strength factor of 5 instead of 1
    # incorporate third sense
    omega += [[0., 0., 0., 0.],
              [0., 0., 0., 0.],
              [0., 0., 5., -5.], 
              [0., 0., -5., 5.]]
    xi += [[0.],
           [0.],
           [-Z2],
           [Z2]]
    
    # display final omega and xi
    print('Omega: \n', omega)
    print('\n')
    print('Xi: \n', xi)
    print('\n')
    
    ## TODO: calculate mu as the inverse of omega * xi
    ## recommended that you use: np.linalg.inv(np.matrix(omega)) to calculate the inverse
    omega_inv = np.linalg.inv(np.matrix(omega))
    mu = omega_inv*xi
    return mu


In [ ]:
# call function and print out `mu`
mu = mu_from_positions(-3, 5, 3, 10, 5, 1)
print('Mu: \n', mu)

In [ ]:
Omega: 
 [[ 3. -1.  0. -1.]
 [-1.  3. -1. -1.]
 [ 0. -1.  6. -5.]
 [-1. -1. -5.  7.]]


Xi: 
 [[-18.]
 [ -3.]
 [  2.]
 [ 16.]]


Mu: 
 [[-3.        ]
 [ 2.32142857]
 [ 6.28571429]
 [ 6.67857143]]